# RSI 70-30 example

- An example strategy [for RSI rebalance](https://help.tokensets.com/en/articles/3541009-eth-btc-rsi-ratio-trading-set-details)

# Set up


In [1]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


# Load data

We use Binance data so we get a longer period of data.

In [2]:
import datetime
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.binance import create_binance_universe

strategy_universe = create_binance_universe(
    ["ETHUSDT", "BTCUSDT"],
    candle_time_bucket=TimeBucket.d1,
    stop_loss_time_bucket=TimeBucket.h4,
    start_at=datetime.datetime(2020, 1, 1),
    end_at=datetime.datetime(2024, 1, 1),
    include_lending=False
)


  0%|          | 0/2 [00:00<?, ?it/s]

# Show loaded trading universe

In [3]:
pairs = strategy_universe.data_universe.pairs  # Trading pairs metadata
candles = strategy_universe.data_universe.candles  # Candles for all trading pairs

print(f"We loaded {candles.get_candle_count():,} candles.")

for pair in pairs.iterate_pairs():
    print(pair)

We loaded 2,924 candles.
<Pair #1 ETH - USDT (0xe82ac67166a910f4092c23f781cd39e46582ec9c) at exchange binance>
<Pair #2 BTC - USDT (0x1d06ef1d6470d25f8e3d6f04f5acc111f176939c) at exchange binance>


# Trading algorithm

In [4]:
from typing import List, Dict

from pandas_ta.overlap import ema
from pandas_ta.momentum import rsi
import pandas as pd

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.state.visualisation import PlotKind, PlotShape
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager

rsi_high = 70
rsi_low = 30
tokens = ["ETH", "BTC"]
lookback_candles = 120


def decide_trades(
        timestamp: pd.Timestamp,
        strategy_universe: TradingStrategyUniverse,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:

    trades = []  # This is the list of trades we are going to  make in  this cycle
    cash = state.portfolio.get_current_cash()  # How much cash we have in a hand

    # Calculate indicators
    for token in tokens:  # Visualise RSI for all of our traded pairs
        pair = pass
        candles = candles.get_single_pair_data(timestamp, sample_count=lookback_candles)

    # Generate trades


    # Plot backtest and live trading visualisation
    visualisation = state.visualisation
    for token in tokens:  # Visualise RSI for all of our traded pairs
        visualisation.plot_indicator(timestamp, f"RSI {token} low", PlotKind.technical_indicator_overlay_on_detached, rsi_low, detached_overlay_name="RSI {token}")
        visualisation.plot_indicator(timestamp, f"RSI {token} high", PlotKind.technical_indicator_overlay_on_detached, rsi_high, detached_overlay_name="RSI {token}")
    return trades

SyntaxError: invalid syntax (4193122099.py, line 33)

# Backtest

In [ ]:
from tradeexecutor.strategy.cycle import CycleDuration

from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    decide_trades=decide_trades,
    name="ETH/USDC fast and slow EMA example",
    client=client,
    cycle_duration=CycleDuration.d1,
    universe=universe,
    # create_trading_universe=create_single_pair_trading_universe,
    initial_deposit=10_000,
    engine_version="0.3",
    universe_options=default_universe_options,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")